In [1]:
import mysql.connector
import pandas as pd

In [2]:
dfCitys = pd.read_excel("./文件集/台灣縣市統整.xlsx")
dfCitys

,地區,縣市,行政區名稱,區號
0,北部,台北市,中正區,100
1,北部,台北市,大同區,103
2,北部,台北市,中山區,104
3,北部,台北市,松山區,105
4,北部,台北市,大安區,106
...,...,...,...,...
366,東部與其他,連江縣,莒光鄉,211
367,東部與其他,連江縣,東引鄉,212
368,東部與其他,南海諸島,東沙群島,817
369,東部與其他,南海諸島,南沙群島,819


In [3]:
dicTW = {}
for area in dfCitys['地區'].unique():
    dicTW[area] = {}
    mask = dfCitys['地區'] == area
    for city in dfCitys[mask]['縣市'].unique():
        dicTW[area][city] = []
        maskCity = dfCitys['縣市'] == city
        for index in dfCitys[maskCity].index:
            zone, zoneNum = dfCitys.loc[index]['行政區名稱'], dfCitys.loc[index]['區號']
            dicTW[area][city].append([zone, zoneNum])

In [4]:
count = 0
for area in dicTW.keys():
    for city in dicTW[area].keys():
        count += len(dicTW[area][city])
count

371

### 建地區database資料

In [12]:
conn = mysql.connector.connect(user='developer1', password='password', host='localhost', database='四技二專')
cursor = conn.cursor()
values = []
for area in dfCitys['地區'].unique():
    values.append((None, area))
sql = "insert into 地區 values (%s, %s)"
cursor.executemany(sql, values)
conn.commit()

### 建縣市database資料

In [79]:
conn = mysql.connector.connect(user='developer1', password='password', host='localhost', database='四技二專')
cursor = conn.cursor()
data = []
for area in dicTW.keys():
    sql = "select 地區ID from 地區 where 地區類型=%s"
    cursor.execute(sql, (area, ))
    areaID = cursor.fetchone()[0]
    for city in dicTW[area].keys():
        data.append((None, city, areaID))
sql = "insert into 縣市 values(%s, %s, %s)"
cursor.executemany(sql, data)
conn.commit()

### 建鄉鎮市區database資料

In [7]:
conn = mysql.connector.connect(user='developer1', password='password', host='localhost', database='四技二專')
cursor = conn.cursor()
data = []
for area in dicTW.keys():
    for city in dicTW[area].keys():
        sql = "select 縣市ID from 縣市 where 縣市名稱=%s"
        cursor.execute(sql, (city,))
        cityID = cursor.fetchone()[0]
        for zone in dicTW[area][city]:
            data.append((None, str(zone[1]), zone[0], cityID))
sql = "insert into 鄉鎮市區 values(%s, %s, %s, %s)"
cursor.executemany(sql, data)
conn.commit()

In [8]:
cursor.close()
conn.close()